In [0]:
CREATE TABLE insurance_data (
    age INTEGER,
    sex VARCHAR(20),
    bmi NUMERIC(8, 3),
    children INTEGER,
    smoker VARCHAR(20),
    region VARCHAR(20),
    charges NUMERIC(10, 4)
);

In [0]:
COPY insurance_data
FROM 's3://wednesday-solutions-intern/data/run-1697288947086-part-r-00000'
IAM_ROLE 'arn:aws:iam::777580185313:role/redshift_assignment_wedsols'
DELIMITER ','
CSV
IGNOREHEADER 1
REGION 'eu-west-1';

In [0]:
SELECT * from insurance_data;

Now we will look at some statistics for AGE,SEX,BMI,CHILDREN,SMOKER,REGION,CHARGES
Going first with AGE

In [0]:
SELECT
  MIN(age) AS min_age,
  MAX(age) AS max_age,
  AVG(age) AS avg_age,
  MEDIAN(age) AS median_age,
  STDDEV(age) AS std_dev_age
FROM insurance_data;

Statistics for SEX

In [0]:
SELECT
    sex,
    COUNT(*) AS total_count,
    (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM insurance_data)) AS percentage
FROM insurance_data
GROUP BY sex;

CHILDREN STATS

In [0]:
SELECT
  MIN(children) AS min_child,
  MAX(children) AS max_child,
  AVG(children) AS avg_child,
  MEDIAN(children) AS median_child
FROM insurance_data;

In [0]:
SELECT
    children,
    COUNT(*) AS children_count
FROM insurance_data
GROUP BY children;

Statistics for BMI

In [0]:
SELECT
  MIN(bmi) AS min_bmi,
  MAX(bmi) AS max_bmi,
  AVG(bmi) AS avg_bmi,
  MEDIAN(bmi) AS median_bmi,
  STDDEV(bmi) AS std_dev_bmi
FROM insurance_data;

SMOKER STATS

In [0]:
SELECT
  smoker,
  COUNT(*) AS count,
 (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM insurance_data))  as percentage
FROM insurance_data
GROUP BY smoker;

REGION STATS

In [0]:
SELECT
    region,
    COUNT(*) AS region_count,
    SUM(charges) AS total_charges
FROM insurance_data
GROUP BY region
ORDER BY region;

CHARGES STATISTICS

In [0]:
SELECT
  MIN(charges) AS min_charges,
  MAX(charges) AS max_charges,
  AVG(charges) AS avg_charges,
  MEDIAN(charges) AS median_charges,
  STDDEV(charges) AS std_dev_charges
FROM insurance_data;

UNDERSTANDING SMOKERS VS NON SMOKERS MEAN AND CUMULATIVE MEDICAL COSTS

In [0]:
select SUM(charges),AVG(CHARGES),sex,smoker from insurance_data GROUP BY sex,smoker;

SO WE CAN SEE THAT  ON AN AVERAGE MALES HAVE ACCUMULATED HIGHER CHARGES CUMULATIVELY AND ON AN AVERAGE TOO AMONGST SMOKERS 
ON THE OTHER HAND IN NON SMOKERS FEMALES HAVE MORE HIGHER AVERAGE/CUMULATIVE CHARGES THAN MEN ...
SO WE CAN UNDERSTAND THAT NON SMOKING RELATED ISSUES ARE MORE IN FEMALES ...
FOR MEN SMOKING RELATED ISSUES ARE MORE

REGION WISE DISTRUBUTION OF DATA FOR THOSE WHO ARE SMOKERS

In [0]:
SELECT SUM(charges),sex,
  region
FROM insurance_data
WHERE smoker='yes'
GROUP BY region,sex;

WE UNDERSTAND BY THIS PLOT THAT THE REGIONS WHERE MALE/FEMALES CAN BE TARGETED BASED ON SMOKING ACTIVITY ...
ESPECIALLY IN THE SOUTHEAST .. MALE SMOKERS ARE MORE SO INSURANCE PREMIUM PLANS SHOULD BE HIGHER HERE...

ANALYSIS ON BMI 

UNDERWEIGHT < 18,

HEALTHY BMI BW 18-24.9

OVERWEIGHT BW 25-29.9

OBESE AFTER THAT

SO WE WILL TRY TO BUILD A COLUMN BMI_STATUS FILLING VALUES ON ABOVE CONDITIONS AND CREATE SOME VISUALS

In [0]:
SELECT SUM(charges),region,
    CASE
        WHEN bmi < 18.5 THEN 'Underweight'
        WHEN bmi >= 18.5 AND bmi <= 24.9 THEN 'Normal'
        WHEN bmi >= 25 AND bmi <= 29.9 THEN 'Overweight'
        ELSE 'Obese'
    END AS bmi_status

    FROM insurance_data GROUP BY region,bmi_status ORDER BY region;

WE OBSERVE HERE THAT THE PEAKS ARE OBSERVED IN OBESE PEOPLE IN ALL THE REGIONS...
THE ORDER GOES LIKE THIS FOR THE CHARGES ACCUMULATED : OBESE>OVERWEIGHT>NORMAL>UNDERWEIGHT

HENCE THE PREMIUM PLANS SHOULD BE MADE MORE EXPENSIVE FOR OBESE AND THEN DECREASE LATER ON .....

LETS ALSO HAVE A LOOK AT SCATTER PLOT

In [0]:
SELECT
    *,
    CASE
        WHEN bmi < 18.5 THEN 'Underweight'
        WHEN bmi >= 18.5 AND bmi <= 24.9 THEN 'Normal'
        WHEN bmi >= 25 AND bmi <= 29.9 THEN 'Overweight'
        ELSE 'Obese'
    END AS bmi_status
FROM insurance_data;

WE CAN SEE HERE BY THIS PLOT THAT OBESE CATEGORY IS IN 

MALES MOSTLY IN THE REGION OF 45-55 AGE BRACKET...

IN FEMALES BY THE SCATTER PLOT WE SEE THAT OBESE MOSTLY LIES IN THE RANGE OF 45-50

LETS MAKE AGE BRACKETS THEN ..

18-33 ,34-50,50 +

THIS SEGREGATION HAS BEEN MADE BY MIN MAX AGE CALCULATION.....

In [0]:
SELECT
    region,
    CASE
        WHEN bmi < 18.5 THEN 'Underweight'
        WHEN bmi >= 18.5 AND bmi <= 24.9 THEN 'Normal'
        WHEN bmi >= 25 AND bmi <= 29.9 THEN 'Overweight'
        ELSE 'Obese'
    END AS bmi_status,
    CASE
        WHEN age BETWEEN 18 AND 33 THEN '18-33'
        WHEN age BETWEEN 34 AND 50 THEN '34-50'
        ELSE '50+'
    END AS age_bracket,
    SUM(charges) AS total_charges
FROM insurance_data
WHERE bmi_status='Obese'
GROUP BY region,bmi_status, age_bracket
ORDER BY region,bmi_status, age_bracket;

THIS PLOT SHOWS US REGION/ AGE BRACKET vs CHARGES ACCUMULATED FOR OBESE PEOPLE ...

FOR ALL REGIONS 50 + AGE BRACKET NEEDS TO HAVE HIGHER PREMIUM ON HEALTH PLAN ..

THIS PLOT CAN BE AUTOMATICALLY CHANGED FOR DIFFERENT BMI_STATUS PEOPLE

ANALYSIS OF SMOKING STATUS WITH REGION AND AGE AND REGION AND BMI

In [0]:
SELECT
    region,
    CASE
        WHEN age BETWEEN 18 AND 33 THEN '18-33'
        WHEN age BETWEEN 34 AND 50 THEN '34-50'
        ELSE '50+'
    END AS age_bracket,
    SUM(charges) AS total_charges
FROM insurance_data
WHERE smoker = 'yes'
GROUP BY region, age_bracket
ORDER BY region, age_bracket;

In [0]:
SELECT
    region,
    CASE
        WHEN bmi < 18.5 THEN 'Underweight'
        WHEN bmi >= 18.5 AND bmi <= 24.9 THEN 'Normal'
        WHEN bmi >= 25 AND bmi <= 29.9 THEN 'Overweight' 
        ELSE 'Obese'
    END AS bmi_status,
    SUM(charges) AS total_charges
FROM insurance_data
WHERE smoker = 'yes'
GROUP BY region, bmi_status
ORDER BY region, bmi_status;

ANALYSIS WITH CHILDREN ...

In [0]:
SELECT
    CASE
        WHEN children < 1 THEN '<1 child'
        WHEN children BETWEEN 1 AND 2 THEN '1-2 children'
        ELSE '>2 children'
    END AS children_interval,
    COUNT(*) AS count,
    SUM(charges) AS total_charges,
    region
FROM insurance_data
GROUP BY children_interval,region
ORDER BY children_interval;

Observing this we can see that People having 1-2 Children are accumulating more Charges. Hence , more premium should be paid for plans having them and Vice versa.